# Does Specializing More than Once Improve Learning
### Without Reg

In [1]:
from res_specialization import *
from matplotlib import pyplot as plt
import sys
import random
import pickle
import warnings

warnings.filterwarnings("ignore", category=FutureWarning) 
plt.rcParams['figure.figsize'] = [10, 5]


In [2]:
TRIALS      = 1000
NUM_TO_SPEC = 3
TOL = 5

diff_eq_params = {"x0": [-20, 10, -.5], 
                  "begin": 0, 
                  "end": 60, 
                  "timesteps":60000}

res_params = {"res_sz": 30, 
              "activ_f": np.tanh,
              "connect_p": .12, 
              "ridge_alpha": .00001, 
              "spect_rad": .9, 
              "gamma": 1., 
              "sigma": 0.12,
              "uniform_weights": True,
              "solver": "least squares"
             }

results = dict()


def save_results():
    pickle.dump(results, open("spec_tw_no_reg.pkl","wb"))
# end

### Model trajectory

In [3]:
t, train_t, u = lorenz_equ(**diff_eq_params )
train_t = t[30000:55000]
test_t  = t[55000:]

### Re-run grant figure trials to get prediction length
(This is fast)


In [4]:
random_err       = []
random_pred      = []
specialized_err  = []
specialized_pred = []


spec_sizes       = []
spec_edges       = []

for i in range(TRIALS):
    
    ######################
    # Random graph
    ######################
    
    # Make rc
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    err = rc.fit(train_t,u)
    
    # Make predicitions
    test_pre = rc.predict(test_t)
    
    # Store Error
    random_err.append(err)
    random_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))
    
    #######################
    # Specialize graph
    ######################
    
    A = specialize_best_nodes(rc, NUM_TO_SPEC, u, train_t, r_0=r_0)
    spec_sizes.append(A.shape[0])
    spec_edges.append(np.sum(A != 0))
    
    rc = make_res_comp(A, res_params)
    r_0 = rc.state_0
    err = rc.fit(train_t,u)
    test_pre = rc.predict(test_t)
    specialized_err.append(err)
    specialized_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))


In [5]:
results["random_err"]       = random_err
results["random_pred"]      = random_pred
results["specialized_err"]  = specialized_err
results["specialized_pred"] = specialized_pred
results["spec_sizes"]       = spec_sizes
results["spec_edges"]       = spec_edges
save_results()
print("Mean Prediction Length: {}".format(np.mean(random_pred)))
print("Mean Prediction Length: {}".format(np.mean(specialized_pred)))

Mean Prediction Length: 572.441
Mean Prediction Length: 812.379


### Control: Same sizes and same edge densities as previous experiment

In [6]:
match_edge_err = []
match_edge_pred = []

for s,e in zip(spec_sizes,spec_edges):
    # Test random graph with same number of edges as specialized graphs
    
    # Make and fit RC
    res_params["res_sz"] = s
    res_params["connect_p"] = e/(s**2)
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    err = rc.fit(train_t,u)
    
    # Predict
    test_pre = rc.predict(test_t)
    match_edge_err.append(err)
    match_edge_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))

In [7]:
results["match_edge_err"] = match_edge_err
results["match_edge_pred"] = match_edge_pred
print("Mean Prediction Length: {}".format(np.mean(match_edge_pred)))
save_results()


Mean Prediction Length: 784.982


### Specialization without finding best nodes

In [8]:
res_params = {"res_sz": 30, 
              "activ_f": np.tanh,
              "connect_p": .12, 
              "ridge_alpha": .00001, 
              "spect_rad": .9, 
              "gamma": 1., 
              "sigma": 0.12,
              "uniform_weights": True,
              "solver": "least squares"
             }

random_specialized_err  = []
random_specialized_pred = []
rspec_sizes         = []
rspec_edges         = []

for i in range(TRIALS):
    # Random graph
    rc = ResComp(3,3,**res_params)
    
    # Turn reservoir to integer adj matrix
    A = rc.res
    for j in range(A.shape[0]): A[j,j] = 0
    A = (A != 0)*1
    
    # Specialize the reservoir
    A = specializeGraph(A, random.sample(list(range(30)),27))
    
    # Store data
    rspec_sizes.append(A.shape[0])
    rspec_edges.append(np.sum(A != 0))
    
    # Make new reservoir
    rc = make_res_comp(A, res_params)
    r_0 = rc.state_0
    err = rc.fit(train_t,u)
    
    # Get predicitons
    test_pre = rc.predict(test_t)
    
    # Store data
    random_specialized_err.append(err)
    random_specialized_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))


In [9]:
results["random_specialized_err"]  = random_specialized_err
results["random_specialized_pred"] = random_specialized_pred
results["rspec_sizes"]             = rspec_sizes
results["rspec_edges"]             = rspec_edges

print("Mean Prediction Length: {}".format(np.mean(random_specialized_pred)))
save_results()


Mean Prediction Length: 793.043


### Control: Same sizes and same edge densities as previous experiment


In [10]:
rand_match_edge_err  = []
rand_match_edge_pred = []

for s,e in zip(rspec_sizes, rspec_edges):
    # Test random graph
    
    # Make and fit RC
    res_params["res_sz"] = s
    res_params["connect_p"] = e/(s**2)
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    err = rc.fit(train_t,u)
    
    # Predict
    test_pre = rc.predict(test_t)
    rand_match_edge_err.append(err)
    rand_match_edge_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))
    

In [11]:
results["rand_match_edge_err"]  = rand_match_edge_err
results["rand_match_edge_pred"] = rand_match_edge_pred
print("Mean Prediction Length: {}".format(np.mean(rand_match_edge_pred)))
save_results()


Mean Prediction Length: 785.206


### Targeted Specialize Twice

In [12]:
res_params = {"res_sz": 30, 
              "activ_f": np.tanh,
              "connect_p": .12, 
              "ridge_alpha": .00001, 
              "spect_rad": .9, 
              "gamma": 1., 
              "sigma": 0.12,
              "uniform_weights": True,
              "solver": "least squares"
             }

spec_tw_pre = []
spec_tw_err = []
tw_edges = []
tw_sizes = []
HOW_MANY_SPEC = 2

for i in range(TRIALS):
    # Make rc
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    err = rc.fit(train_t,u)
    
    # Specialize 2 times
    for i in range(HOW_MANY_SPEC):
        A = specialize_best_nodes(rc, NUM_TO_SPEC, u, train_t, r_0=r_0)
        rc = make_res_comp(A, res_params)
        r_0 = rc.state_0
        rc.fit(train_t,u)
    
    # Predict the system states
    tw_sizes.append(A.shape[0])
    tw_edges.append(np.sum(A != 0))
    test_pre = rc.predict(test_t)
    spec_tw_err.append(err)
    spec_tw_pre.append(how_long_accurate(u(test_t), test_pre,tol=TOL))

In [13]:
results["spec_tw_pre"] = spec_tw_pre
results["spec_tw_err"] = spec_tw_err
results["tw_edges"] = tw_edges
results["tw_sizes"] = tw_sizes
save_results()


In [14]:
print("Mean Prediction Length: {}".format(np.mean(spec_tw_pre)))


Mean Prediction Length: 834.564


### Control: Same sizes and same edge densities as previous experiment

In [15]:
tw_match_edge_err  = []
tw_match_edge_pred = []

for s,e in zip(tw_sizes, tw_edges):
    # Test random graph
    
    # Make and fit RC
    res_params["res_sz"] = s
    res_params["connect_p"] = e/(s**2)
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    err = rc.fit(train_t,u)
    
    # Predict
    test_pre = rc.predict(test_t)
    tw_match_edge_err.append(err)
    tw_match_edge_pred.append(how_long_accurate(u(test_t),test_pre,tol=TOL))
    

In [16]:
results["tw_match_edge_err"] = tw_match_edge_err
results["tw_match_edge_pred"] = tw_match_edge_pred
save_results()
print("Mean Prediction Length: {}".format(np.mean(tw_match_edge_pred)))


Mean Prediction Length: 796.686


### Targeted Specialize Three Times

In [17]:
res_params = {"res_sz": 30, 
              "activ_f": np.tanh,
              "connect_p": .12, 
              "ridge_alpha": .00001, 
              "spect_rad": .9, 
              "gamma": 1., 
              "sigma": 0.12,
              "uniform_weights": True,
              "solver": "least squares"
             }

spec_thr_pre = []
spec_thr_err = []
thr_edges = []
thr_sizes = []
HOW_MANY_SPEC = 3

for i in range(TRIALS):
    # Make RC
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    err = rc.fit(train_t,u)
    
    # Specialize 3 times
    for i in range(HOW_MANY_SPEC):
        A = specialize_best_nodes(rc, NUM_TO_SPEC, u, train_t, r_0=r_0)
        rc = make_res_comp(A, res_params)
        r_0 = rc.state_0
        rc.fit(train_t,u)
    
    # Make predicitons
    thr_sizes.append(A.shape[0])
    thr_edges.append(np.sum(A != 0))
    test_pre = rc.predict(test_t)
    spec_thr_err.append(err)
    spec_thr_pre.append(how_long_accurate(u(test_t), test_pre,tol=TOL))


/usr/local/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


In [18]:
results["spec_thr_pre"] = spec_thr_pre
results["spec_thr_err"] = spec_thr_err
results["thr_edges"] = thr_edges
results["thr_sizes"] = thr_sizes
print("Mean Prediction Length: {}".format(np.mean(spec_thr_pre)))

save_results()


Mean Prediction Length: 823.73


### Control: Same sizes and same edge densities as previous experiment

In [19]:
thr_match_edge_err  = []
thr_match_edge_pred = []

for s,e in zip(thr_sizes, thr_edges):
    # Test random graph
    
    # Make and fit RC
    res_params["res_sz"] = s
    res_params["connect_p"] = e/(s**2)
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    err = rc.fit(train_t,u)
    
    # Predict systems
    test_pre = rc.predict(test_t)
    thr_match_edge_err.append(err)
    thr_match_edge_pred.append(how_long_accurate(u(test_t),test_pre,tol=TOL))
    

In [20]:
results["thr_match_edge_err"] = thr_match_edge_err
results["thr_match_edge_pred"] = thr_match_edge_pred
print("Mean Prediction Length: {}".format(np.mean(thr_match_edge_pred)))

save_results()


Mean Prediction Length: 785.494


## Organize results into dataframes


In [21]:
import pandas as pd

In [22]:
labels = ["random_specialized","specialized","spec_twice", "spec_three"]
df = pd.DataFrame(results)
df.describe()

,random_err,random_pred,specialized_err,specialized_pred,spec_sizes,spec_edges,match_edge_err,match_edge_pred,random_specialized_err,random_specialized_pred,...,tw_edges,tw_sizes,tw_match_edge_err,tw_match_edge_pred,spec_thr_pre,spec_thr_err,thr_edges,thr_sizes,thr_match_edge_err,thr_match_edge_pred
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.997531,572.441000,0.139404,812.379000,93.80200,200.348000,0.170815,784.982000,0.175890,793.043000,...,254.677000,126.186000,0.091188,796.686000,823.730000,0.992530,302.849000,153.948000,0.057343,785.494000
std,0.195415,151.784383,0.149714,224.061784,62.21818,104.454927,0.178557,206.991916,0.167247,177.325902,...,161.204173,92.210174,0.106923,262.169843,343.834051,0.194921,191.370094,105.595434,0.072603,288.591662
min,0.437225,11.000000,0.001158,76.000000,33.00000,79.000000,0.001274,67.000000,0.004348,194.000000,...,86.000000,36.000000,0.000254,44.000000,38.000000,0.465643,99.000000,41.000000,0.000246,23.000000
25%,0.860976,498.000000,0.034417,720.000000,58.00000,141.000000,0.040723,697.000000,0.054340,702.000000,...,166.000000,75.000000,0.019672,729.000000,757.750000,0.856078,188.000000,89.000000,0.012506,732.750000
50%,0.978147,573.000000,0.089339,810.500000,75.00000,173.000000,0.114144,782.000000,0.121692,783.000000,...,209.000000,99.000000,0.052249,803.000000,836.500000,0.978614,246.000000,122.000000,0.029926,807.000000
75%,1.121450,651.500000,0.188203,898.000000,108.00000,224.000000,0.235051,866.250000,0.244803,875.000000,...,281.250000,143.000000,0.120420,884.000000,909.250000,1.118852,358.000000,187.000000,0.072582,880.000000
max,1.727015,1227.000000,0.888094,4389.000000,655.00000,1161.000000,1.145017,4157.000000,1.073294,3638.000000,...,1945.000000,1104.000000,0.805706,4959.000000,5000.000000,1.865011,1971.000000,1063.000000,0.535584,5000.000000


### Error

In [23]:
df[["random_specialized_err","specialized_err","spec_tw_err", "spec_thr_err"]].describe()

,random_specialized_err,specialized_err,spec_tw_err,spec_thr_err
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.175890,0.139404,1.005789,0.992530
std,0.167247,0.149714,0.199986,0.194921
min,0.004348,0.001158,0.527981,0.465643
25%,0.054340,0.034417,0.865608,0.856078
50%,0.121692,0.089339,0.992053,0.978614
75%,0.244803,0.188203,1.123624,1.118852
max,1.073294,0.888094,1.759101,1.865011


### Error Controls

In [24]:
df[["rand_match_edge_err", "match_edge_err", "tw_match_edge_err", "thr_match_edge_err"]].describe()

,rand_match_edge_err,match_edge_err,tw_match_edge_err,thr_match_edge_err
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.212432,0.170815,0.091188,0.057343
std,0.185496,0.178557,0.106923,0.072603
min,0.004574,0.001274,0.000254,0.000246
25%,0.072382,0.040723,0.019672,0.012506
50%,0.160856,0.114144,0.052249,0.029926
75%,0.297409,0.235051,0.120420,0.072582
max,1.262886,1.145017,0.805706,0.535584


### Prediction

In [25]:
df[["random_specialized_pred","specialized_pred","spec_tw_pre", "spec_thr_pre"]].describe()

,random_specialized_pred,specialized_pred,spec_tw_pre,spec_thr_pre
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,793.043000,812.379000,834.564000,823.730000
std,177.325902,224.061784,363.609202,343.834051
min,194.000000,76.000000,35.000000,38.000000
25%,702.000000,720.000000,751.000000,757.750000
50%,783.000000,810.500000,826.000000,836.500000
75%,875.000000,898.000000,904.250000,909.250000
max,3638.000000,4389.000000,5000.000000,5000.000000


### Prediction Control

In [26]:
df[["rand_match_edge_pred", "match_edge_pred", "tw_match_edge_pred", "thr_match_edge_pred"]].describe()

,rand_match_edge_pred,match_edge_pred,tw_match_edge_pred,thr_match_edge_pred
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,785.206000,784.982000,796.686000,785.494000
std,269.608751,206.991916,262.169843,288.591662
min,215.000000,67.000000,44.000000,23.000000
25%,687.000000,697.000000,729.000000,732.750000
50%,767.000000,782.000000,803.000000,807.000000
75%,848.000000,866.250000,884.000000,880.000000
max,4944.000000,4157.000000,4959.000000,5000.000000


### Sizes

In [27]:
sizes = df[["rspec_sizes", "spec_sizes", "tw_sizes", "thr_sizes"]]
sizes.describe()

,rspec_sizes,spec_sizes,tw_sizes,thr_sizes
count,1000.000000,1000.00000,1000.000000,1000.000000
mean,76.999000,93.80200,126.186000,153.948000
std,37.184912,62.21818,92.210174,105.595434
min,27.000000,33.00000,36.000000,41.000000
25%,53.000000,58.00000,75.000000,89.000000
50%,67.500000,75.00000,99.000000,122.000000
75%,90.000000,108.00000,143.000000,187.000000
max,351.000000,655.00000,1104.000000,1063.000000


### Edges

In [28]:
edges = df[["rspec_edges", "spec_edges", "tw_edges", "thr_edges"]]
edges.describe()

,rspec_edges,spec_edges,tw_edges,thr_edges
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,176.630000,200.348000,254.677000,302.849000
std,64.652076,104.454927,161.204173,191.370094
min,73.000000,79.000000,86.000000,99.000000
25%,135.000000,141.000000,166.000000,188.000000
50%,162.000000,173.000000,209.000000,246.000000
75%,198.000000,224.000000,281.250000,358.000000
max,605.000000,1161.000000,1945.000000,1971.000000


### Connectivity: (num edges)/(num nodes)^2

In [29]:
labels = ["random_specialized","specialized","spec_twice", "spec_three"]
edge_prob = pd.DataFrame(np.array(edges)/(np.array(sizes)**2),columns=labels)
edge_prob.describe()

,random_specialized,specialized,spec_twice,spec_three
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.037537,0.031813,0.022733,0.017931
std,0.016939,0.016480,0.011877,0.009697
min,0.004886,0.002706,0.001596,0.001743
25%,0.025106,0.019278,0.013599,0.010497
50%,0.035510,0.030215,0.021228,0.016386
75%,0.047707,0.041320,0.029692,0.023938
max,0.117717,0.098255,0.072531,0.061868


In [30]:
df[["random_err", "random_pred"]].describe()

,random_err,random_pred
count,1000.000000,1000.000000
mean,0.997531,572.441000
std,0.195415,151.784383
min,0.437225,11.000000
25%,0.860976,498.000000
50%,0.978147,573.000000
75%,1.121450,651.500000
max,1.727015,1227.000000
